##画像を20種類に分類するタスクです
https://signate.jp/competitions/108

In [17]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

from torchvision.datasets import ImageFolder
from torchvision import transforms
import sys,os,shutil

In [2]:
label_data=pd.read_csv("label_master.tsv",sep="\t",header=0)
label_data

,label_id,label_name
0,0,aquatic_mammals
1,1,fish
2,2,flowers
3,3,food_containers
4,4,fruit_and_vegetables
5,5,household_electrical_devices
6,6,household_furniture
7,7,insects
8,8,large_carnivores
9,9,large_man-made_outdoor_things


In [3]:
train_label=pd.read_csv("train_master.tsv",sep='\t',header=0)
train_label.head()

,file_name,label_id
0,train_00000.png,11
1,train_00001.png,15
2,train_00002.png,4
3,train_00003.png,14
4,train_00004.png,1


In [19]:
#train_dataをtrain_dataとval_dataに分ける（提出用のテストデータは教師データがない）

modes=["train", "val"]
os.makedirs(r"C:\Users\heste\machine_learning\pytorch\ImagePractice\val",exist_ok=True)
dgpath=r"C:\Users\heste\machine_learning\pytorch\ImagePractice"

for i in range(40000,50000):
    img_path="train_{0:05d}.png".format(i)
    a=os.path.join(dgpath,modes[0],img_path)
    b=os.path.join(dgpath,modes[1],img_path)
    shutil.move(a,b)
    

In [20]:
#trainとvalに0-19のラベルフォルダーを作る

lalabel=list(train_label['label_id'])
for mode in modes:   
    for i in range(20):
        path=r"{}\{}".format(mode,i)
        os.makedirs(path,exist_ok=True)
        

In [21]:
#画像をラベルフォルダーに格納する


dgpath=r"C:\Users\heste\machine_learning\pytorch\ImagePractice"
for mode in modes:
    if mode=='train':
        for i in range(40000):
            img_path="train_{0:05d}.png".format(i)
            label_path="{}".format(lalabel[i])
            first_=os.path.join(dgpath, mode, img_path)
            next_=os.path.join(dgpath, mode, label_path, img_path)
            shutil.move(first_, next_)
    else:
        for i in range(40000,50000):
            img_path="train_{0:05d}.png".format(i)
            label_path="{}".format(lalabel[i])
            first_=os.path.join(dgpath, mode, img_path)
            next_=os.path.join(dgpath, mode, label_path, img_path)
            shutil.move(first_, next_)

In [78]:
#画像のデータセットを作る


sys.path.append(os.pardir)

# ImageFolder関数を使用してDatasetを作成する
train_imgs = ImageFolder(
    "train/",
    transform=transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
))
val_imgs = ImageFolder(
    "val/",
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
))

# DataLoaderを作成
train_loader = DataLoader(
    train_imgs, batch_size=40, shuffle=False)
val_loader = DataLoader(
    val_imgs, batch_size=40, shuffle=False)

In [5]:
#転移学習の準備
from torchvision import models
print(models.resnet50())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

)


In [6]:
#モデルの作成
from torchvision import models

# 事前学習済みのresnet18をロード
net = models.resnet50(pretrained=True)

# すべてのパラメータを微分対象外にする
for p in net.parameters():
    p.requires_grad=False
    
# 最後の線形層を付け替える
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 20)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\heste/.cache\torch\checkpoints\resnet50-19c8e357.pth


①学習時にconv層毎回計算するタイプ

In [93]:
#学習の補助関数作成
def eval_net(net, data_loader, device="cpu"):
    # networkを評価モードにする
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # toメソッドで計算を実行するデバイスに転送する
        x = x.to(device)
        y = y.to(device)
        
        # 確率が最大のクラスを予測
        # ここではforward（推論）の計算だけなので微分を保存しない
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # ミニバッチごとの予測結果などを1つにまとめる
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 予測精度を計算
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

def train_net(net, train_loader, test_loader,
              only_fc=True,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        # 最後の線形層のパラメータのみを、
        # optimizerに渡す
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # ネットワークを訓練モードにする
        net.train()
        n = 0
        n_acc = 0
                
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 訓練データの予測精度
        train_acc.append(n_acc / n)
        # 検証データの予測精度
        val_acc.append(eval_net(net, test_loader, device))
        # このepochでの結果を表示
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [ ]:
# ネットワークの全パラメータをGPUに転送
net#.to("cuda:0")

# 訓練を実行
train_net(net, train_loader, val_loader, n_iter=20)# device="cuda:0")

①の学習結果、CrossEntropyLoss=2.14, train_acc=0.35, val=acc=0.36 程度でした。あまり学習できていないです。

②conv層の計算をあらかじめしておいて、学習時は全結合層のみ計算するタイプ

In [9]:
class IdentityLayer(nn.Module):
    def forward(self, x):
        return x
    
cal_net = models.resnet50(pretrained=True)
for p in cal_net.parameters():
    p.requires_grad=False
cal_net.fc = IdentityLayer()


In [69]:
#あらかじめ学習済みResNet18のconv層による計算をしておき、それを画像データの代わりに特徴量とする

class PreCalDataset(Dataset):
    
    def __init__(self,root,transform):
        self.root=root
        self.transform=transform
        self.ImDataset=ImageFolder(self.root, self.transform)
        self.datanum=len(self.ImDataset)
        
    def __len__(self):
        return self.datanum
        
        
    def __getitem__(self,idx):
        print("4545 for debug")
        x,y = self.dataset[idx]
        x=torch.tensor([x]).float()
        with torch.no_grad():
            new_x= cal_net(x).detach()
        
        return new_x, y
        

In [79]:
#newデータの用意
#PreCaldatasetの__getitem__が上手くいかないのでこのcellは意味を成しません
train_new_imgs=PreCalDataset(
    "train/",
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
)

val_new_imgs=PreCalDataset(
    "val/",
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
)

train_new_loader=DataLoader(train_new_imgs, batch_size=40, shuffle=True)
val_new_loader=DataLoader(val_new_imgs, batch_size=40, shuffle=False)


In [94]:
#PreCalDatasetがうまくいかないので、繰り返し処理で画像（３，３２，３２）を特徴量ベクトル（2048、）に変える

def im2vec(Image_loader,device="cpu"):
    cal_net.to(device)
    cal_net.eval()
    new_x_list=[]
    y_list=[]
    
    for x, y in tqdm.tqdm(Image_loader):
        x, y= x.to(device), y.to(device)
        with torch.no_grad():
            new_x_list.append(cal_net(x).detach())
            y_list.append(y)
    
    new_x_tensor=torch.cat(new_x_list)
    y_tensor=torch.cat(y_list)
    new_dataset=TensorDataset(new_x_tensor, y_tensor)
    return new_dataset

In [95]:
##newデータローダの用意
train_new_imgs= im2vec(train_loader)
val_new_imgs= im2vec(val_loader)

train_new_loader=DataLoader(train_new_imgs, batch_size=40, shuffle=True)
val_new_loader=DataLoader(val_new_imgs, batch_size=40, shuffle=False)



  0%|                                                                                                                              | 0/1000 [00:00<?, ?it/s]

  0%|                                                                                                                      | 1/1000 [00:00<03:11,  5.23it/s]

  0%|▏                                                                                                                     | 2/1000 [00:00<03:18,  5.02it/s]

  0%|▎                                                                                                                     | 3/1000 [00:00<03:14,  5.13it/s]

  0%|▍                                                                                                                     | 4/1000 [00:00<03:15,  5.09it/s]

  0%|▌                                                                                                                     | 5/1000 [00:01<03:20,  4.96it/s]

  1%|▋                                            

  5%|█████▉                                                                                                               | 51/1000 [00:11<03:38,  4.34it/s]

  5%|██████                                                                                                               | 52/1000 [00:11<03:39,  4.32it/s]

  5%|██████▏                                                                                                              | 53/1000 [00:11<03:26,  4.59it/s]

  5%|██████▎                                                                                                              | 54/1000 [00:12<03:30,  4.50it/s]

  6%|██████▍                                                                                                              | 55/1000 [00:12<03:28,  4.54it/s]

  6%|██████▌                                                                                                              | 56/1000 [00:12<03:23,  4.63it/s]

  6%|██████▋                                        

 10%|███████████▊                                                                                                        | 102/1000 [00:22<03:54,  3.82it/s]

 10%|███████████▉                                                                                                        | 103/1000 [00:23<04:15,  3.51it/s]

 10%|████████████                                                                                                        | 104/1000 [00:23<04:14,  3.51it/s]

 10%|████████████▏                                                                                                       | 105/1000 [00:23<04:12,  3.55it/s]

 11%|████████████▎                                                                                                       | 106/1000 [00:23<04:00,  3.72it/s]

 11%|████████████▍                                                                                                       | 107/1000 [00:24<03:46,  3.93it/s]

 11%|████████████▌                                  

 15%|█████████████████▋                                                                                                  | 153/1000 [00:36<03:45,  3.76it/s]

 15%|█████████████████▊                                                                                                  | 154/1000 [00:36<03:39,  3.85it/s]

 16%|█████████████████▉                                                                                                  | 155/1000 [00:37<03:41,  3.82it/s]

 16%|██████████████████                                                                                                  | 156/1000 [00:37<03:34,  3.93it/s]

 16%|██████████████████▏                                                                                                 | 157/1000 [00:37<03:58,  3.53it/s]

 16%|██████████████████▎                                                                                                 | 158/1000 [00:38<04:16,  3.28it/s]

 16%|██████████████████▍                            

 20%|███████████████████████▋                                                                                            | 204/1000 [00:50<03:28,  3.82it/s]

 20%|███████████████████████▊                                                                                            | 205/1000 [00:50<03:26,  3.86it/s]

 21%|███████████████████████▉                                                                                            | 206/1000 [00:51<03:25,  3.87it/s]

 21%|████████████████████████                                                                                            | 207/1000 [00:51<03:24,  3.87it/s]

 21%|████████████████████████▏                                                                                           | 208/1000 [00:51<03:26,  3.83it/s]

 21%|████████████████████████▏                                                                                           | 209/1000 [00:51<03:24,  3.88it/s]

 21%|████████████████████████▎                      

...

  8%|████████▉                                                                                                             | 19/250 [00:04<00:56,  4.09it/s]

  8%|█████████▍                                                                                                            | 20/250 [00:04<00:57,  3.99it/s]

  8%|█████████▉                                                                                                            | 21/250 [00:05<00:57,  3.99it/s]

  9%|██████████▍                                                                                                           | 22/250 [00:05<00:56,  4.00it/s]

  9%|██████████▊                                                                                                           | 23/250 [00:05<00:56,  3.99it/s]

 10%|███████████▎                                                                                                          | 24/250 [00:05<00:57,  3.93it/s]

 10%|███████████▊                                   

 28%|█████████████████████████████████                                                                                     | 70/250 [00:17<00:46,  3.91it/s]

 28%|█████████████████████████████████▌                                                                                    | 71/250 [00:17<00:45,  3.92it/s]

 29%|█████████████████████████████████▉                                                                                    | 72/250 [00:18<00:44,  3.98it/s]

 29%|██████████████████████████████████▍                                                                                   | 73/250 [00:18<00:45,  3.93it/s]

 30%|██████████████████████████████████▉                                                                                   | 74/250 [00:18<00:44,  3.94it/s]

 30%|███████████████████████████████████▍                                                                                  | 75/250 [00:18<00:43,  4.00it/s]

 30%|███████████████████████████████████▊           

 48%|████████████████████████████████████████████████████████▋                                                            | 121/250 [00:30<00:31,  4.13it/s]

 49%|█████████████████████████████████████████████████████████                                                            | 122/250 [00:30<00:30,  4.24it/s]

 49%|█████████████████████████████████████████████████████████▌                                                           | 123/250 [00:30<00:30,  4.11it/s]

 50%|██████████████████████████████████████████████████████████                                                           | 124/250 [00:30<00:30,  4.15it/s]

 50%|██████████████████████████████████████████████████████████▌                                                          | 125/250 [00:31<00:29,  4.19it/s]

 50%|██████████████████████████████████████████████████████████▉                                                          | 126/250 [00:31<00:29,  4.14it/s]

 51%|███████████████████████████████████████████████

 69%|████████████████████████████████████████████████████████████████████████████████▍                                    | 172/250 [00:42<00:19,  4.05it/s]

 69%|████████████████████████████████████████████████████████████████████████████████▉                                    | 173/250 [00:43<00:18,  4.06it/s]

 70%|█████████████████████████████████████████████████████████████████████████████████▍                                   | 174/250 [00:43<00:18,  4.02it/s]

 70%|█████████████████████████████████████████████████████████████████████████████████▉                                   | 175/250 [00:43<00:18,  4.09it/s]

 70%|██████████████████████████████████████████████████████████████████████████████████▎                                  | 176/250 [00:43<00:18,  3.95it/s]

 71%|██████████████████████████████████████████████████████████████████████████████████▊                                  | 177/250 [00:44<00:18,  3.89it/s]

 71%|███████████████████████████████████████████████

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 223/250 [00:55<00:06,  4.01it/s]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 224/250 [00:55<00:06,  4.01it/s]

 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▎           | 225/250 [00:55<00:06,  4.16it/s]

 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 226/250 [00:56<00:05,  4.12it/s]

 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 227/250 [00:56<00:05,  4.08it/s]

 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▋          | 228/250 [00:56<00:05,  3.98it/s]

 92%|███████████████████████████████████████████████

In [96]:
#ロジスティック回帰のモデリングから学習まで
linear_net = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 20),
                                 nn.LogSoftmax(dim=1))

train_net(linear_net, train_new_loader, val_new_loader, only_fc=False, loss_fn=nn.NLLLoss(), n_iter=20)
    
    



  0%|                                                                                                                              | 0/1000 [00:00<?, ?it/s]

  0%|▍                                                                                                                     | 4/1000 [00:00<00:29, 33.38it/s]

  1%|█▏                                                                                                                   | 10/1000 [00:00<00:26, 37.96it/s]

  2%|█▊                                                                                                                   | 16/1000 [00:00<00:23, 41.54it/s]

  2%|██▌                                                                                                                  | 22/1000 [00:00<00:21, 44.76it/s]

  3%|███▎                                                                                                                 | 28/1000 [00:00<00:20, 48.12it/s]

  3%|███▉                                         

 30%|██████████████████████████████████▊                                                                                 | 300/1000 [00:05<00:12, 55.36it/s]

 31%|███████████████████████████████████▍                                                                                | 306/1000 [00:05<00:12, 54.46it/s]

 31%|████████████████████████████████████▏                                                                               | 312/1000 [00:05<00:12, 55.70it/s]

 32%|████████████████████████████████████▉                                                                               | 318/1000 [00:06<00:12, 54.41it/s]

 32%|█████████████████████████████████████▌                                                                              | 324/1000 [00:06<00:12, 54.21it/s]

 33%|██████████████████████████████████████▎                                                                             | 330/1000 [00:06<00:12, 52.98it/s]

 34%|███████████████████████████████████████        

 61%|██████████████████████████████████████████████████████████████████████▉                                             | 612/1000 [00:11<00:09, 42.61it/s]

 62%|███████████████████████████████████████████████████████████████████████▌                                            | 617/1000 [00:11<00:08, 42.68it/s]

 62%|████████████████████████████████████████████████████████████████████████▏                                           | 622/1000 [00:11<00:08, 42.52it/s]

 63%|████████████████████████████████████████████████████████████████████████▋                                           | 627/1000 [00:11<00:08, 42.51it/s]

 63%|█████████████████████████████████████████████████████████████████████████▎                                          | 632/1000 [00:12<00:08, 42.61it/s]

 64%|█████████████████████████████████████████████████████████████████████████▉                                          | 637/1000 [00:12<00:08, 42.79it/s]

 64%|███████████████████████████████████████████████

 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 890/1000 [00:17<00:02, 48.52it/s]

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 895/1000 [00:17<00:02, 48.72it/s]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████▍           | 900/1000 [00:17<00:02, 43.82it/s]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉           | 905/1000 [00:17<00:02, 40.66it/s]

 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌          | 910/1000 [00:17<00:02, 38.99it/s]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏         | 915/1000 [00:18<00:02, 40.91it/s]

 92%|███████████████████████████████████████████████

0 1.7284849920311012 0.4509 0.5085999965667725


...



  0%|                                                                                                                              | 0/1000 [00:00<?, ?it/s]

  1%|▋                                                                                                                     | 6/1000 [00:00<00:19, 49.88it/s]

  1%|█▎                                                                                                                   | 11/1000 [00:00<00:20, 48.61it/s]

  2%|█▊                                                                                                                   | 16/1000 [00:00<00:20, 48.80it/s]

  2%|██▍                                                                                                                  | 21/1000 [00:00<00:19, 49.10it/s]

  3%|███                                                                                                                  | 26/1000 [00:00<00:20, 47.32it/s]

  3%|███▋                                         

 27%|███████████████████████████████▎                                                                                    | 270/1000 [00:05<00:14, 49.49it/s]

 28%|████████████████████████████████                                                                                    | 276/1000 [00:05<00:15, 47.91it/s]

 28%|████████████████████████████████▌                                                                                   | 281/1000 [00:05<00:14, 48.26it/s]

 29%|█████████████████████████████████▎                                                                                  | 287/1000 [00:06<00:14, 49.34it/s]

 29%|█████████████████████████████████▊                                                                                  | 292/1000 [00:06<00:14, 47.35it/s]

 30%|██████████████████████████████████▍                                                                                 | 297/1000 [00:06<00:14, 47.83it/s]

 30%|███████████████████████████████████▏           

 54%|██████████████████████████████████████████████████████████████▊                                                     | 541/1000 [00:11<00:09, 48.02it/s]

 55%|███████████████████████████████████████████████████████████████▎                                                    | 546/1000 [00:11<00:09, 47.28it/s]

 55%|████████████████████████████████████████████████████████████████                                                    | 552/1000 [00:11<00:08, 49.83it/s]

 56%|████████████████████████████████████████████████████████████████▋                                                   | 558/1000 [00:11<00:09, 48.30it/s]

 56%|█████████████████████████████████████████████████████████████████▎                                                  | 563/1000 [00:11<00:08, 48.67it/s]

 57%|█████████████████████████████████████████████████████████████████▉                                                  | 568/1000 [00:11<00:08, 48.69it/s]

 57%|███████████████████████████████████████████████

 81%|██████████████████████████████████████████████████████████████████████████████████████████████▍                     | 814/1000 [00:17<00:03, 49.74it/s]

 82%|███████████████████████████████████████████████████████████████████████████████████████████████                     | 820/1000 [00:17<00:03, 48.12it/s]

 82%|███████████████████████████████████████████████████████████████████████████████████████████████▋                    | 825/1000 [00:17<00:03, 48.56it/s]

 83%|████████████████████████████████████████████████████████████████████████████████████████████████▎                   | 830/1000 [00:17<00:03, 46.79it/s]

 84%|████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 836/1000 [00:17<00:03, 48.26it/s]

 84%|█████████████████████████████████████████████████████████████████████████████████████████████████▌                  | 841/1000 [00:17<00:03, 48.44it/s]

 85%|███████████████████████████████████████████████

19 0.5321873337537557 0.811675 0.5507000088691711


②の学習結果は、loss=0.53, train_acc=0.81, val_acc=0.55
学習が上手くいったが、若干過学習気味ではある

In [103]:
#提出用データのサンプル
pd.read_csv("sample_submit.csv",header=None).head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,test_00000.png,0.036151,0.014607,0.034591,0.079934,0.053778,0.011690,0.039329,0.079885,0.047027,...,0.062593,0.037947,0.080989,0.066377,0.020849,0.057315,0.056351,0.033405,0.078505,0.053430
1,test_00001.png,0.033393,0.037818,0.021027,0.043661,0.082885,0.041105,0.069309,0.059735,0.023879,...,0.077569,0.053666,0.044468,0.040490,0.090077,0.050215,0.052718,0.067940,0.030642,0.078394
2,test_00002.png,0.098902,0.047091,0.022525,0.059422,0.038273,0.011877,0.086344,0.054882,0.046612,...,0.028493,0.027183,0.092749,0.076549,0.026914,0.099406,0.032409,0.038710,0.000611,0.060666
3,test_00003.png,0.054705,0.080344,0.047048,0.083701,0.043293,0.068830,0.026523,0.076391,0.008649,...,0.020802,0.054486,0.065079,0.060174,0.045692,0.052243,0.006735,0.036344,0.070731,0.026432
4,test_00004.png,0.087472,0.045188,0.091187,0.003265,0.110322,0.102048,0.048273,0.080005,0.050142,...,0.096636,0.038774,0.050342,0.044085,0.057429,0.023512,0.018324,0.002596,0.030469,0.001106


In [121]:
#testデータに対して上記のようなデータフレームを作る
from PIL import Image

array_2d=[]

#①で学習したならTrue,②ならFalse
Fit_1=False
if Fit_1:
    p_fn=nn.Softmax()
else:
    p_fn=nn.Softmax()
    net=nn.Sequential(
        cal_net,
        linear_net
    )

for i in range(10000):
    im_path="test_{0:05d}.png".format(i)
    path=r"C:\Users\heste\machine_learning\pytorch\ImagePractice\test\test_{0:05d}.png".format(i)
    
    im=Image.open(path)
    p=np.asarray(im, dtype=float)
    #netはバッチでデータを受け取るので一番外側にリストを作る
    p=torch.tensor([p.transpose(2,0,1)], requires_grad=False)
    #tensorはdouble型になっているのでfloat型に直す
    p=p.float()
    net.eval()
    value=list(p_fn(net(p)).detach().numpy().squeeze())
    value.insert(0,im_path)
    array_2d.append(value)

sub_df=pd.DataFrame(array_2d)


C:\Users\heste\.conda\envs\pyenvML\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [122]:
sub_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,test_00000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,test_00001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,test_00002.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,test_00003.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,test_00004.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,test_00005.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,test_00006.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,test_00007.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,test_00008.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,test_00009.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [138]:
#submitファイル作成
sub_df.to_csv('2020-03-27submit.csv',header=None,index=None)